In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_cleaned_v2.csv')
print(tweets_df.shape)
tweets_df.head()

(17913, 4)


,tweet_id,tweet,city,year
0,0,bad thing say government medical spare operati...,Mumbai,2022
1,1,netas family admitted government hospital priv...,Mumbai,2022
2,2,govandi one hotspot well respiratory disease d...,Mumbai,2022
3,3,till medical negligence exist government hospi...,Mumbai,2022
4,4,doctor reading also government hospital resident,Mumbai,2022
